# Usando el autologger para no batallar

## Pruebas sobre el uso de MLFlow

### Aprendizaje Automático Aplicado

![](https://mcd.unison.mx/wp-content/themes/awaken/img/logo_mcd.png)

**Julio Waissman**

## 1. Abriendo MLFlow y creando un experimento de chocolate

Aqui en esta sección vamos a conctarnos al servidor de MLFlow y vamos a crear un experimento y una entrada de chocolate, solo para probar. Si ya existía el experimento (*chocolate* lo vamos a llamar) pues lo abre, y si no lo crea. 

Empecemos por cargar mlflow y generar el experimento

In [ ]:
!pip install mlflow

In [ ]:
import mlflow

# Conectandose a MLFlow
mlflow.set_tracking_uri("http://nodo3:45974") 


# Generando el experimento o cargandolo si existe
experiment_name = "prueba"
mlflow.set_experiment(experiment_name)

# Cargando la información
client = mlflow.tracking.MlflowClient()
experiment_id = client.get_experiment_by_name(experiment_name).experiment_id

# Vamos a ver si es cierto
print(f"MLflow Version: {mlflow.__version__}")
print(f"Tracking URI: {mlflow.tracking.get_tracking_uri()}")
print(f"Nombre del experimento: {experiment_name}")
print(f"ID del experimento: {experiment_id}")



y ahora si vamos a generar datos para guardar la info de chocolate

In [ ]:

# Vamos a usar las clases Param Metric y RunTag para ver opciones de registo
from mlflow.entities import Param, Metric, RunTag

def run(alpha, nombre_corrida, tiempo):
    with mlflow.start_run(run_name=nombre_corrida) as run:
        
        # Guardando parámetros, métricas y tags de uno por uno
        # Son inventados pues
        mlflow.log_param("alpha", str(alpha))
        mlflow.log_metric("rmse", 0.666)
        mlflow.set_tag("nombre_corrida", nombre_corrida)
        
        # Creando y guardando como artefato un archivo txt
        with open("toto.txt", "w") as f:
            f.write(f"Artifacto generado para {nombre_corrida}")
        mlflow.log_artifact("toto.txt")
        
        # Como generar parametros, metricas y tags y guardarlas de un jalón
        params = [ Param("p1","0.1"), Param("p2","0.2") ]
        metrics = [ Metric("m1", 0.1, tiempo, 0), Metric("m2", 0.2, tiempo, 0) ]
        tags = [ RunTag("t1", "hi1"), RunTag("t2", "hi2") ]
        client.log_batch(run.info.run_uuid, metrics, params, tags)
        
        # Para ver que es lo que estamos guardando
        print(f"Id: {run.info.run_uuid}")
        print(f"artifact_uri: {mlflow.get_artifact_uri()}")
        print(f"alpha: {alpha}")
        print(f"nombre de la corrida:", nombre_corrida)


In [ ]:
# Vamos a guardar la fecha
import time
now = round(time.time())

run(3.1416, 'chocolata', now)

## 2. Usando autologger con sklearn

Vamos a ver como funciona el autologger usando un `grid_seach` desde la librería `sklearn` para esto, vamos a utilizar el ejemplo de aprendizaje que [viene en la documentación de grid seach de scikit-learn](https://scikit-learn.org/stable/auto_examples/model_selection/grid_search_text_feature_extraction.html#sphx-glr-auto-examples-model-selection-grid-search-text-feature-extraction-py), pero registrandola en un experimento de MLFlow. Así podemos ver las ventajas de utilizar MLFlow para registrar los diferentes experimentos aunque sea en un problema de chocolate.

Empecemos por crear el experimento


In [ ]:
# Generando el experimento o cargandolo si existe
experiment_name = "noticias"
mlflow.set_experiment(experiment_name)

# Cargando la información
client = mlflow.tracking.MlflowClient()
experiment_id = client.get_experiment_by_name(experiment_name).experiment_id

# Vamos a ver si es cierto
print(f"MLflow Version: {mlflow.__version__}")
print(f"Tracking URI: {mlflow.tracking.get_tracking_uri()}")
print(f"Nombre del experimento: {experiment_name}")
print(f"ID del experimento: {experiment_id}")

Ahora vamos a descargar los datos

In [ ]:
from sklearn.datasets import fetch_20newsgroups

categories = [
    "alt.atheism",
    "talk.religion.misc",
]
# Uncomment the following to do the analysis on all the categories
# categories = None

data = fetch_20newsgroups(subset="train", categories=categories)

print(f"{len(data.filenames)} documentos")
print(f"{len(data.target_names)} categorías")

Por fin, el aprendizaje, tal como viene en la dicumentación, pero con mlflow

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

def entrena(data, parametros=None):
    mlflow.sklearn.autolog()
    pipeline = Pipeline(
        [
            ("vect", CountVectorizer()),
            ("tfidf", TfidfTransformer()),
            ("clf", SGDClassifier()),
        ]
    )

    if parametros == None:
        parametros = {
            "vect__max_df": (0.5, 0.75, 1.0),
            "vect__ngram_range": ((1, 1), (1, 2)),  # unigrams or bigrams
            "clf__max_iter": (20,),
            "clf__alpha": (0.00001, 0.000001),
            "clf__penalty": ("l2",),
        }
    
    # Define y entrena con el autologger prendido
    grid_search = GridSearchCV(pipeline, parametros, n_jobs=-1)
    grid_search.fit(data.data, data.target)
        

In [ ]:
# uncommenting more parameters will give better exploring power but will
# increase processing time in a combinatorial way
parametros = {
    "vect__max_df": (0.5, 0.75, 1.0),
    "vect__max_features": (None, 5000, 10000, 50000),
    "vect__ngram_range": ((1, 1), (1, 2)),  # unigrams or bigrams
    "tfidf__use_idf": (True, False),
    #"tfidf__norm": ('l1', 'l2'),
    "clf__alpha": (0.00001, 0.000001),
    "clf__penalty": ("l2", "elasticnet"),
    #"clf__max_iter": (20,),
    "clf__max_iter": (20, 50, 80),
}

entrena(data, parametros)
    
# Imprimiendo donde se encuentra la información
run_id = mlflow.last_active_run().info.run_id
print(f"Datos y modelo en la corrida: {run_id}")


## 2. Autologer con Keras con un modelo desde cero

Ahora vamos a hacer un modelito por default de los que usa uno cuando está aprendiando Keras y Tensorflow con el autologger, para ver como funciona de bonito. 

Empecemos bajando los datos de MINST

In [ ]:
import pandas as pd
import numpy as np
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

(x_train, y_train), (x_test, y_test) = mnist.load_data()

def reshape(x, n):
    x = x.reshape((n, 28 * 28))
    return x.astype('float32') / 255

x_train = reshape(x_train, x_train.shape[0])
x_test = reshape(x_test, x_test.shape[0])
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)


print("Data after reshape:")
print("  x_train.shape:", x_train.shape)
print("  y_train.shape:", y_train.shape)
print("  x_test.shape:", x_test.shape)
print("  y_test.shape:", y_test.shape)

y ahora creemos o abramos el experimento

In [ ]:
# Generando el experimento o cargandolo si existe
experiment_name = "minst-keras"
mlflow.set_experiment(experiment_name)

# Cargando la información
client = mlflow.tracking.MlflowClient()
experiment_id = client.get_experiment_by_name(experiment_name).experiment_id

# Vamos a ver si es cierto
print(f"MLflow Version: {mlflow.__version__}")
print(f"Tracking URI: {mlflow.tracking.get_tracking_uri()}")
print(f"Nombre del experimento: {experiment_name}")
print(f"ID del experimento: {experiment_id}")

y ahora las funciones para el aprendizaje

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
import mlflow.keras
import mlflow.tensorflow

def entrena_keras_MINST(x_train, y_train, x_test, y_test, 
                        epochs, batch_size, seed=42, autolog='keras'):

    # Para reproducibilkidad, si seed es None, no se pone
    if seed != None:
        np.random.seed(seed)
        tf.random.set_seed(seed)
    
    if autolog == 'keras':
        mlflow.keras.autolog()
    elif autolog == 'tensorflow':
        mlflow.tensorflow.autolog()
    else:
        mlflow.autolog()

    with mlflow.start_run() as run:
        mlflow.set_tag("version.mlflow", mlflow.__version__)
        mlflow.set_tag("version.keras", keras.__version__)
        mlflow.set_tag("version.tensorflow", tf.__version__)
        mlflow.set_tag("autolog", autolog)
        mlflow.set_tag("seed", f"{seed}")
        
        # Un modelito bien simple solo por probar
        model = keras.models.Sequential()
        model.add(keras.layers.Dense(512, activation="relu", input_shape=(28 * 28,)))
        model.add(keras.layers.Dense(10, activation="softmax"))
        
        # Compilando y entrenando
        model.compile(
            optimizer="rmsprop",
            loss="categorical_crossentropy",
            metrics=["accuracy"]
        )
        model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0)
        test_loss, test_acc = model.evaluate(x_test, y_test)
        
        # Guardando el modelo en formato json
        with open("model.json", "w") as f:
            f.write(model.to_json())
        mlflow.log_artifact("model.json")
        mlflow.log_metrics(
            {'test_loss': test_loss, 'test_acc':test_acc}
        )
        

        

y ejecutamos

In [ ]:
epochs = 5
batch_size = 32

entrena_keras_MINST(
    x_train, y_train, x_test, y_test, 
    epochs, batch_size, seed=42,
    autolog='tensorflow'
)
